# Test 1

### Import

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from backtesting import Backtest
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os
from datetime import datetime
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting

### Device

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Using device: {device}")

### Hyperparameter

In [ ]:
# Model parameter
input_size = 10
output_size = 1
hidden_size = 1000
num_layers = 6
dropout = 0.2

# Training parameter
batch_size = 16
num_epochs = 50
learning_rate = 0.001
seq_size = 30

### LSTM Model

In [ ]:
class Net(nn.Module):

    def __init__(self, input_size, output_size, hidden_size, num_layers):
        super(Net, self).__init__()
        self.layer_1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.layer_2 = nn.LSTM(hidden_size, hidden_size, batch_first=True, dropout=dropout, num_layers=num_layers)
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.layer_1(x)  
        x = torch.relu(x)

        x, _ = self.layer_2(x)
        x = torch.relu(x)

        x = self.output_layer(x[:, -1, :])  
        
        return x

### Dataloader

In [ ]:
class FinanceDataset(Dataset):
    def __init__(self, data_path, seq_size):
        self.data = pd.read_pickle(data_path)
        self.seq_size = seq_size

        self.data = self.data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'month', 'weekday', 'INTEREST_RATE', 'trend', 'Invest']]

        # Entfernen der 'Date'-Spalte für Inputs
        self.inputs = self.data.drop(columns=['Invest']).values
        self.labels = self.data['Invest'].values

        # Skalieren der Eingabedaten
        scaler = MinMaxScaler()
        self.inputs = scaler.fit_transform(self.inputs)

    def __len__(self):
        return len(self.inputs) - self.seq_size

    def __getitem__(self, idx):
        # Slice für Sequenzeingaben und das entsprechende Label
        x = self.inputs[idx:idx + self.seq_size]
        y = self.labels[idx + self.seq_size]

        # In Tensoren umwandeln
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)

        return x, y
    

### Init

In [ ]:
# Initialize model, loss function, optimizer
net = Net(input_size, output_size, hidden_size, num_layers)
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate)

In [ ]:
# Initialize dataset and dataloader
dataset = FinanceDataset('../Data/train_dax_data.pkl', seq_size=seq_size)
train_loader = DataLoader(dataset, batch_size=batch_size)

### Training

In [ ]:
os.makedirs('../Models', exist_ok = True)
losses = []
# Training loop
for epoch in range(num_epochs):
    running_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = net(inputs.float())
        loss = criterion(torch.squeeze(outputs), labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # Save model after each epoch
    model_path = f'../Models/model-{epoch + 1}.pt'
    torch.save(net.state_dict(), model_path)

    
    if epoch % 10 == 0:
        learning_rate *= 0.8
        optimizer = optim.SGD(net.parameters(), lr=learning_rate)

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')
    losses.append(running_loss / len(train_loader))



os.makedirs("../Models", exist_ok=True)

model_path = "../Models/best_model.pt"

if os.path.exists(model_path):
    net.load_state_dict(torch.load(model_path))
    print(f"Modell erfolgreich geladen von {model_path}")


# Parameter für Early Stopping und Modell-Speicherung
patience = 5 
best_test_loss = float('inf')
early_stopping_counter = 0

losses = []
test_loss_vals = []
learning_rate = 0.001  # Initiale Lernrate

# Training loop mit Early Stopping
for epoch in range(num_epochs):
    net.train()
    running_loss = 0

    # Training
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Auf das Gerät verschieben
        optimizer.zero_grad()
        outputs = net(inputs.float())
        loss = criterion(torch.squeeze(outputs), labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    losses.append(avg_train_loss)

    # Modell auswerten auf dem Testset
    net.eval()
    running_test_loss = 0
    with torch.no_grad():
        # Iteriere über den Testloader
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Auf das Gerät verschieben
            test_outputs = net(inputs)
            test_outputs = test_outputs.squeeze(-1)  # Entferne unnötige Dimension
            test_loss = criterion(test_outputs, labels)
            running_test_loss += test_loss.item()
        
    avg_test_loss = running_test_loss / len(test_loader)
    test_loss_vals.append(avg_test_loss)

    # Check for Early Stopping und Speichern des besten Modells
    if avg_test_loss < best_test_loss:
        best_test_loss = avg_test_loss
        best_model_path = '../Models/best_model.pt'
        torch.save(net.state_dict(), best_model_path)
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    # Ausgabe der Verluste
    print(f'Epoch {epoch + 1}, Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}')
    
    # Early Stopping-Kriterium prüfen
    if early_stopping_counter >= patience:
        print(f"Early stopping after {epoch + 1} epochs. Test loss did not improve for {patience} epochs.")
        break

    # Dynamische Lernratenanpassung alle 10 Epochen
    if epoch > 0 and epoch % 10 == 0:
        learning_rate *= 0.5
        optimizer = optim.SGD(net.parameters(), lr=learning_rate)
        print(f"Reduced learning rate to {learning_rate:.5f}")

print(f"Training abgeschlossen. Bestes Modell gespeichert unter: {best_model_path}")

### Backtesting

In [ ]:

scaler = MinMaxScaler()
df = pd.read_pickle('../Data/train_dax_data.pkl').reset_index()
df = df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'month', 'weekday', 'INTEREST_RATE', 'trend']]
scaler.fit(df.values)

alredy_done = [f.removeprefix("backtest-").removesuffix(".csv.gz") for f in os.listdir("results/")]

test_data = pd.read_pickle("../Data/test_dax_data.pkl")
models = [f for f in os.listdir("../Models/") if os.path.isfile(os.path.join("../Models/", f))]
results = []

os.makedirs("logs", exist_ok=True)
os.makedirs("results", exist_ok=True)

for model_name in models:

    model_path = f"../Models/{model_name}"
    model = Net(input_size, output_size, hidden_size, num_layers)

    # Load state_dict only
    model.load_state_dict(torch.load(model_path, weights_only=False))
    model.eval()

    ALPACA_CREDS = {
        "API_KEY": os.getenv("ALPACA_API_KEY"), 
        "API_SECRET": os.getenv("ALPACA_API_SECRET"), 
        "PAPER": True
    }

    # Strategy setup
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 12, 31)
    broker = Alpaca(ALPACA_CREDS)

    # Instantiate and run the strategy
    strategy = Backtest(
        name=model_name,  
        broker=broker,
        parameters={
            "symbol": "^GDAXI",
            "cash_at_risk": 0.5,
            "model": model,
            "num_prior_days": 30,
            "dataset": test_data,
            "scaler": scaler,
        }
    )

    # Run the backtest
    backtest_results = strategy.backtest(
        YahooDataBacktesting,
        start_date,
        end_date,
        parameters={"symbol": "^GDAXI", "cash_at_risk": 0.5, "model": model, "dataset": test_data, "scaler": scaler},
        benchmark_asset="^GDAXI",
        plot_file_html=f"results/plot-{str(model_name)}.html",
        tearsheet_file=f"results/tearsheet-{model_name}.html",
        trades_file=f"results/plot-{str(model_name)}",
        settings_file=f"results/plot-{str(model_name)}",
        indicators_file=f"results/plot-{str(model_name)}",
        show_plot=True,
        show_tearsheet=True
    )
    backtest_results = pd.DataFrame(backtest_results)
    backtest_results["model"] = model_name
    results.append(backtest_results)

# Save the results to a CSV file
pd.concat(results, ignore_index=True).to_csv("results.csv", index=False)

print("Backtesting complete. Results saved to backtest_results.csv.")


In [ ]:
df = pd.read_csv("results.csv")
display(df.sort_values(by=["model"]))

In [ ]:

best_model = df[df["total_return"] == df["total_return"].max()]
best_model.reset_index(inplace=True)

model_name = best_model.at[0, "model"]
print(f"Best model: {model_name}")

In [ ]:
import pandas as pd
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
import numpy as np

model_path = f"../Models/{model_name}"
model = Net(input_size, output_size, hidden_size, num_layers)

# Load state_dict only
model.load_state_dict(torch.load(model_path))  # Do not use weights_only
model.eval()

# Load your DataFrame
data = pd.read_pickle('../Data/test_dax_data.pkl')

test_data = FinanceDataset('../Data/test_dax_data.pkl', seq_size=seq_size)
test_loader = DataLoader(test_data, batch_size=batch_size)
# Select all rows and columns from index 1 to -1 (exclusive)

all_predictions = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        out = model(inputs) 
        # Store predictions and labels
        all_predictions.append(out.numpy())  
        all_labels.append(labels.numpy())

# Concatenate results
all_predictions = np.concatenate(all_predictions)
all_labels = np.concatenate(all_labels)

# Print or analyze the predictions
print(f'Predicted values: {all_predictions.flatten()}')
print(f'Actual values: {all_labels.flatten()}')

output_df = pd.DataFrame({'Predicted': all_predictions.flatten(), 'Actual': all_labels.flatten()})


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

threshold = 0.5
predicted_classes = (all_predictions.flatten() > threshold).astype(int)

accuracy = accuracy_score(all_labels.flatten(), predicted_classes)
precision = precision_score(all_labels.flatten(), predicted_classes)
recall = recall_score(all_labels.flatten(), predicted_classes)
f1 = f1_score(all_labels.flatten(), predicted_classes)

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(all_labels.flatten(), predicted_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
output_df = pd.DataFrame({
    'Predicted': all_predictions.flatten(),
    'Predicted_Class': predicted_classes,
    'Actual': all_labels.flatten()
})
correlation_matrix = output_df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()
display(output_df)